# JUICE RPWI HF SID2 (RAW): L1a Mask Formation for ASW1/2/3 -- 2025/10/14

# Import lib (same with "sample")

In [ ]:
import copy
import csv
import datetime
import math
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import sys

# Library setting (same with "sample")

In [ ]:
# The latest CDF library https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/
from spacepy import pycdf
import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf39_1-dist/lib"

# *** Library ***
sys.path.append('./lib/')
import juice_cal_lib   as juice_cal
import juice_cdf_lib   as juice_cdf
# import juice_sid2_data as juice_data
import juice_sid2_lib  as juice_sid2
import juice_spec_lib  as juice_spec

# Mode seting (same with "sample")

In [ ]:
# *** Plot dump ***
dump_mode = 0                           # 0: no-dump  1:plot dump  2:comparison for operation
# *** CAL ***
unit_mode = 0                           # [Power]     0: raw     1: V＠ADC     2: V@HF    3: V@RWI  4: V/m
band_mode = 0                           # [Power]     0: sum     1: /Hz
cal_mode  = 2                           # [Power]     0: background     1: CAL     2: all
comp_mode = 0                           # [Complex]   0: Poweer  1: Matrix   3: Matrix-2D  >3: any
# *** Unit ***
f_mode    = 1                           # [Frequency] 0: linear  1: log
time_mode = 1                           # [Time]      0: Epoch   1: data number
gap_mode  = 0                           # [gap]       0: n/a     1: gap
# *** Frenquency in Linear ***
f_mode_min = 80;  f_mode_max = 2068     # 80 - 2068 : as same as SID-4/20
# *** Power range ***
p_raw_max = 7.0                         # background: 7.5   CAL: 10
p_raw_min = 0.0                         # background: 2.5   CAL: 5
# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/JUICE_python/ql/'   # Plot dump folder

In [ ]:
E_min = '2024-08-19 21:05:00';  t_min0 = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S")
E_max = '2024-08-19 21:10:00';  t_max0 = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S")
t_min0 = 0;  t_max0 = 0

#### (special)

In [ ]:
# *** Filter make modes ***
asw_ver      = 3.2  # 1.0 / 2.0 / 3.0 / 3.1 / 3.2  for mask making
sid3_diff    = 2    # [dB]
filter_mode  = 0    # 0: read   1: make MASK        2: make debug output    "lib/hf_sid3_mask.csv"
filter_sid3  = 1    # 0: n/a    1: make freq table                          "lib/hf_sid3_freq.csv"

# **** Mask file set ***
n_freq_msk = 19392;  f_min_msk = 20.0;   f_step_msk = 2.3125

# get CDF data (same with "sample")

In [ ]:
# date='20230419';  ver = 'V03'
# date='20230530';  ver = 'V03'   # CAL
# date='20230601';  ver = 'V03'
# date='20230712';  ver = 'V03'
# date='20230713';  ver = 'V03'
# date='20240125';  ver = 'V02'
# date='20240822';  ver = 'V02'   # CAL
# date='20250331';  ver = 'V01'   # CAL
date='0';         ver = 'V02'   # Test data
data_dir, data_name_list = juice_sid2.datalist(date, ver)       # [date]   yyyymmdd: group read    others: file list

In [ ]:
class struct:
    pass

data = struct()
num_list = len(data_name_list)
for i in range(num_list):
    data_name = data_name_list[i];  cdf_file = data_dir + data_name;  print(i, cdf_file)
    # RPWI_FSW_version = juice_cdf._RPWI_FSW_version(cdf_file)
    cdf = pycdf.CDF(cdf_file);      data1 = juice_sid2.hf_sid2_read(cdf) # , RPWI_FSW_version)
    if i==0:
        data = data1;                                print(data.Eu_i.shape)
    else:
        data = juice_sid2.hf_sid2_add(data, data1);  print(data.Eu_i.shape)
data_name = os.path.split(data_name)[1];             print("data name:", data_name)

In [ ]:
data = juice_sid2.hf_sid2_shaping(data, cal_mode)

In [ ]:
date1 = data.epoch[0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[-1]; date2 = date2.strftime('%Y/%m/%d %R:%S')
str_date = date1 + "  -  " + date2
f_min0  = data.frequency[0][0][0]; f_max0  = max(np.ravel(data.frequency))

print("       date and time:", str_date)
print("         Num-samples:", data.n_time, "  Num-Frequency:", data.n_step, "  Length:", data.n_samp)
print("         Time-length:", data.time[0][0][-1], "sec in 1-sweep, with time resolution of ", data.time[0][0][1], "sec")
print("f, width, step (kHz):", f_min0,  "-", f_max0, data.freq_width[0][0][0], data.freq_step[0][0][0])
print("                  Ch:", data.ch_selected[data.n_time//2])

# Wave data (same with "sample")

In [ ]:
T_HF  = data.T_HF_FPGA[data.n_time//2];  T_RWI = (data.T_RWI_CH1[data.n_time//2] + data.T_RWI_CH2[data.n_time//2])/2
if T_HF  > 199 or T_HF  < -50  or math.isnan(T_HF):
   T_HF  = 25;  data.T_HF_FPGA[:] = math.nan
if T_RWI > 199 or T_RWI < -199 or math.isnan(T_RWI):
   T_RWI = 25;  data.T_RWI_CH1[:] = math.nan;  data.T_RWI_CH2[:] = math.nan

print("Temperature: ", 'HF - {:.1f}'.format(T_HF), '  RWI1 - {:.1f}'.format(T_RWI))

In [ ]:
wave = juice_cal.wave_cal(data, 2, unit_mode, T_HF, T_RWI)
p_max0 = p_raw_max + wave.cf/10;     p_min0 = p_raw_min + wave.cf/10
# p_max0 = np.ceil(np.log10( np.nanmax( [np.nanmax(wave.Eu_i), np.nanmax(wave.Ev_i), np.nanmax(wave.Ew_i), np.nanmax(wave.Eu_q), np.nanmax(wave.Ev_q), np.nanmax(wave.Ew_q)] ) )*5)/5+.5

print("unit_mode:", unit_mode, "  conversion factor:", '{:.1f}'.format(wave.cf), 
      "  MAX-min:", '{:.1f}'.format(p_max0), '{:.1f}'.format(p_min0), "  str_unit:", wave.str_unit, 
      "  T-HF & T-RWI:", T_HF, "(", data.T_HF_FPGA[data.n_time//2], ")", T_RWI, "(", data.T_RWI_CH1[data.n_time//2], ")")

In [ ]:
print("decimation:", juice_cdf._sample_rate(data.decimation[0]))
print("Peak Eu:", '{:.2e}'.format(np.nanmax(wave.Eu_i)), ' {:.2e}'.format(np.nanmax(wave.Eu_q)), wave.str_unit)
print("Peak Ev:", '{:.2e}'.format(np.nanmax(wave.Ev_i)), ' {:.2e}'.format(np.nanmax(wave.Ev_q)), wave.str_unit)
print("Peak Ew:", '{:.2e}'.format(np.nanmax(wave.Ew_i)), ' {:.2e}'.format(np.nanmax(wave.Ew_q)), wave.str_unit)
if unit_mode==0:
    print("Peak Eu:", '{:x}'.format(np.int64(np.nanmax(wave.Eu_i))), ' {:x}'.format(np.int64(np.nanmax(wave.Eu_q))), wave.str_unit)
    print("Peak Ev:", '{:x}'.format(np.int64(np.nanmax(wave.Ev_i))), ' {:x}'.format(np.int64(np.nanmax(wave.Ev_q))), wave.str_unit)
    print("Peak Ew:", '{:x}'.format(np.int64(np.nanmax(wave.Ew_i))), ' {:x}'.format(np.int64(np.nanmax(wave.Ew_q))), wave.str_unit)

In [ ]:
fig = plt.figure(figsize=(16, 11))
ax1 = fig.add_subplot(6, 1, 1);  ax2 = fig.add_subplot(6, 1, 2);  ax3 = fig.add_subplot(6, 1, 3)
ax4 = fig.add_subplot(6, 1, 4);  ax5 = fig.add_subplot(6, 1, 5);  ax6 = fig.add_subplot(6, 1, 6)

ax1.plot(np.ravel(wave.Eu_i[:][:]), '-r', linewidth=.5, label='Eu_i'); ax1.plot(np.ravel(wave.Eu_q[:][:]), ':g', linewidth=.5, label='Eu_q')
ax2.plot(np.ravel(wave.Ev_i[:][:]), '-r', linewidth=.5, label='Ev_i'); ax2.plot(np.ravel(wave.Ev_q[:][:]), ':g', linewidth=.5, label='Ev_q')
ax3.plot(np.ravel(wave.Ew_i[:][:]), '-r', linewidth=.5, label='Ew_i'); ax3.plot(np.ravel(wave.Ew_q[:][:]), ':g', linewidth=.5, label='Ew_q')
ax4.plot(np.ravel(data.frequency),     '-b', linewidth=.5,  label='Frequency')
ax4.plot(np.ravel(data.freq_step*10),  '-g', linewidth=0.8, label='step*10')
ax4.plot(np.ravel(data.freq_width*10), ':b', linewidth=1.0, label='width*10')
ax5.plot(np.ravel(data.T_HF_FPGA),  ':r', label='T (HK-FPGA)');  ax5.plot(np.ravel(data.T_RWI_CH1),     ':b', label='T (RWI1)');  
ax5.plot(np.ravel(data.T_RWI_CH2),  ':g', label='T (RWI2)');     ax5.plot(np.ravel(data.cal_signal*10), '-k', label='CAL-Singal')
ax6.plot(np.ravel(data.epoch[:]), '.')

ax1.set_ylabel('Eu '+wave.str_unit);     ax2.set_ylabel('Ev '+wave.str_unit);  ax3.set_ylabel('Ew '+wave.str_unit);  
ax4.set_ylabel('Frequency [kHz]');  ax5.set_ylabel('T [degC]');      ax6.set_xlabel(str_date)
title_label = '[JUICE/RPWI HF RAW (SID-2)]\n' + data_name;           ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8)

xlim=[-0.5, len(np.ravel(data.Eu_i[:][:])) -0.5];  ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)
xlim=[-0.5, data.n_time                        -0.5];  ax5.set_xlim(xlim);  ax6.set_xlim(xlim)
p_max = np.ceil(np.log10(np.nanmax([np.nanmax(wave.Eu_i), np.nanmax(wave.Eu_q), np.nanmax(wave.Ev_i), np.nanmax(wave.Ev_q), np.nanmax(wave.Ew_i), np.nanmax(wave.Ew_q)]))*5)/5
ylim=[-10**(p_max), 10**(p_max)];  ax1.set_ylim(ylim);  ax2.set_ylim(ylim);  ax3.set_ylim(ylim)
ylim=[f_min0, f_max0];             ax4.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw.png'
    fig.savefig(png_fname)

### WAVE - First 3 Sweeps (same with "sample")

In [ ]:
num_sweep = 3;  n_sweep0 = data.n_time//2;  n_sweep1 = n_sweep0 + num_sweep;  print("[specific sweep]  SWEEP:", n_sweep0, n_sweep1, "in total of", data.n_time)
f_min = f_min0;  f_max = f_max0

fig = plt.figure(figsize=(16, 11))
ax1 = fig.add_subplot(5, 1, 1);  ax2 = fig.add_subplot(5, 1, 2);  ax3 = fig.add_subplot(5, 1, 3);  ax4 = fig.add_subplot(5, 1, 4);  ax5 = fig.add_subplot(5, 1, 5)

ax1.plot(np.ravel(wave.Eu_i[n_sweep0:n_sweep1]), '-r', linewidth=.5, label='Eu_i');  ax1.plot(np.ravel(wave.Eu_q[n_sweep0:n_sweep1]), ':g', linewidth=.5, label='Eu_q')
ax2.plot(np.ravel(wave.Ev_i[n_sweep0:n_sweep1]), '-r', linewidth=.5, label='Ev_i');  ax2.plot(np.ravel(wave.Ev_q[n_sweep0:n_sweep1]), ':g', linewidth=.5, label='Ev_q')
ax3.plot(np.ravel(wave.Ew_i[n_sweep0:n_sweep1]), '-r', linewidth=.5, label='Ew_i');  ax3.plot(np.ravel(wave.Ew_q[n_sweep0:n_sweep1]), ':g', linewidth=.5, label='Ew_q')
ax4.plot(np.ravel(data.frequency[n_sweep0:n_sweep1]),     '-b', linewidth=.5,  label='Frequency')
ax4.plot(np.ravel(data.freq_step[n_sweep0:n_sweep1]*10),  '-g', linewidth=0.8, label='step*10')
ax4.plot(np.ravel(data.freq_width[n_sweep0:n_sweep1]*10), ':b', linewidth=1.0, label='width*10')
ax5.plot(np.ravel(data.epoch[n_sweep0:n_sweep1+1]), '.')

date1 = data.epoch[n_sweep0];  date1 = date1.strftime('%Y/%m/%d %R:%S');  ax5.set_xlabel(date1)
ax1.set_ylabel('Eu '+wave.str_unit);  ax2.set_ylabel('Ev '+wave.str_unit);  ax3.set_ylabel('Ew '+wave.str_unit);  ax4.set_ylabel('Frequency [kHz]');  
title_label = '[JUICE/RPWI HF RAW (SID-2)]\n' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8)

xlim=[-0.5, len(np.ravel(data.Eu_i[n_sweep0:n_sweep1])) -0.5]; ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim)
xlim=[-0.5, num_sweep                                   -0.5]; ax5.set_xlim(xlim)
p_max = np.ceil(np.log10( np.nanmax( [np.nanmax(wave.Eu_i[n_sweep0:n_sweep1]), np.nanmax(wave.Eu_q[n_sweep0:n_sweep1]), np.nanmax(wave.Ev_i[n_sweep0:n_sweep1]), np.nanmax(wave.Ev_q[n_sweep0:n_sweep1]), np.nanmax(wave.Ew_i[n_sweep0:n_sweep1]), np.nanmax(wave.Ew_q[n_sweep0:n_sweep1])] ) )*5)/5
ylim=[-10**(p_max), 10**(p_max)]; ax1.set_ylim(ylim);  ax2.set_ylim(ylim);  ax3.set_ylim(ylim)
ylim=[f_min, f_max]; ax4.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw-sweep.png'
    fig.savefig(png_fname)

### WAVE - First 3 steps (same with "sample")

In [ ]:
num_step = 3;  n_sweep = data.n_time//2;  n_step0 = data.n_step//2;  n_step1 = n_step0 + num_step;  print("[specific sweep]  SWEEP:", n_sweep, "in total of", data.n_time,  "[Steps]", n_step0, "-", n_step1)
fig = plt.figure(figsize=(16, 11))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(np.ravel(wave.Eu_i[n_sweep][n_step0:n_step1]), '-r', linewidth=.5, label='Eu_i');  ax1.plot(np.ravel(wave.Eu_q[n_sweep][n_step0:n_step1]), ':g', linewidth=.5, label='Eu_q')
ax2.plot(np.ravel(wave.Ev_i[n_sweep][n_step0:n_step1]), '-r', linewidth=.5, label='Ev_i');  ax2.plot(np.ravel(wave.Ev_q[n_sweep][n_step0:n_step1]), ':g', linewidth=.5, label='Ev_q')
ax3.plot(np.ravel(wave.Ew_i[n_sweep][n_step0:n_step1]), '-r', linewidth=.5, label='Ew_i');  ax3.plot(np.ravel(wave.Ew_q[n_sweep][n_step0:n_step1]), ':g', linewidth=.5, label='Ew_q')
ax4.plot(np.ravel(data.frequency[n_sweep][n_step0:n_step1]),     '-b', linewidth=.5, label='Frequency')

date1 = data.epoch[n_sweep];  date1 = date1.strftime('%Y/%m/%d %R:%S');  ax4.set_xlabel(date1)
ax1.set_ylabel('Eu '+wave.str_unit);  ax2.set_ylabel('Ev '+wave.str_unit);  ax3.set_ylabel('Ew '+wave.str_unit);  ax4.set_ylabel('Frequency [kHz]');  
title_label = '[JUICE/RPWI HF RAW (SID-2)]\n' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8)

xlim=[-0.5, len(np.ravel(data.Eu_i[n_sweep][n_step0:n_step1])) -0.5];  ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)
p_max = np.ceil(np.log10( np.nanmax( [np.nanmax(wave.Eu_i[n_sweep][n_step0:n_step1]), np.nanmax(wave.Eu_q[n_sweep][n_step0:n_step1]), np.nanmax(wave.Ev_i[n_sweep][n_step0:n_step1]), np.nanmax(wave.Ev_q[n_sweep][n_step0:n_step1]), np.nanmax(wave.Ew_i[n_sweep][n_step0:n_step1]), np.nanmax(wave.Ew_q[n_sweep][n_step0:n_step1])] ) )*5)/5
ylim=[-10**(p_max), 10**(p_max)]; ax1.set_ylim(ylim);  ax2.set_ylim(ylim);  ax3.set_ylim(ylim)

# Plot
fig.subplots_adjust(hspace=0)
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw-step.png'
    fig.savefig(png_fname)

# Spectrum Data (same with "sample")

In [ ]:
spec = juice_spec.hf_getspec_sid2  (data)                               # Wave -> Spectrum
spec = juice_cal.spec_cal(spec, 2, unit_mode, band_mode, T_HF, T_RWI)   # CAL
spec.EE = spec.EuEu + spec.EvEv + spec.EwEw

spec.epoch = spec.epoch.tolist();               spec.n_time = spec.EuEu.shape[0];  
num_1d    = np.arange(spec.n_time)
freq_1d   = spec.freq  [spec.n_time//2];  n_freq1 = freq_1d.shape[0];    f_min1 = freq_1d[0];  f_max1  = freq_1d[-1]
freq_w_1d = spec.freq_w[spec.n_time//2]

In [ ]:
p_max1 = p_raw_max + spec.cf/10;  p_min1 = p_raw_min + spec.cf/10
# p_max1 = np.ceil(np.log10( np.nanmax( [np.nanmax(spec.EuEu), np.nanmax(spec.EvEv), np.nanmax(spec.EwEw)] ) )*5)/5+.5
# p_min1 = np.ceil(np.log10( np.nanmin( [np.nanmin(spec.EuEu), np.nanmin(spec.EvEv), np.nanmin(spec.EwEw)] ) )*5)/5+.5

print(spec.EuEu.shape, "[frequency]", f_min1, "-", f_max1, "kHz  (df =", freq_w_1d[0], "-", freq_w_1d[-1], "kHz)")
print(spec.str_unit, "(unit_mode:", unit_mode, ")  conversion-factor:", '{:.1f}'.format(spec.cf), "[wave:", '{:.1f}'.format(wave.cf),
      "]  T-HF & T-RWI:", T_HF, "(", data.T_HF_FPGA[data.n_time//2], ")", T_RWI, "(", data.T_RWI_CH1[data.n_time//2], ")")
print("[MAX-min]", '{:.1f}'.format(p_max1), "(", '{:.1f}'.format(p_max0), ')  {:.1f}'.format(p_min1), "(", '{:.1f}'.format(p_min0), ")")

In [ ]:
print("Peak   Eu:", '{:.2e}'.format(np.nanmax(wave.Eu_i)), ' {:.2e}'.format(np.nanmax(wave.Eu_q)), wave.str_unit)
print("Peak   Ev:", '{:.2e}'.format(np.nanmax(wave.Ev_i)), ' {:.2e}'.format(np.nanmax(wave.Ev_q)), wave.str_unit)
print("Peak   Ew:", '{:.2e}'.format(np.nanmax(wave.Ew_i)), ' {:.2e}'.format(np.nanmax(wave.Ew_q)), wave.str_unit)
#
peak_E = np.ravel(spec.EuEu[:][:]); peak_f = np.nanargmax(peak_E)
print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), wave.str_unit, '  |  {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])), spec.str_unit, ' {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])**.5), wave.str_unit)
peak_E = np.ravel(spec.EvEv[:][:]); peak_f = np.nanargmax(peak_E)
print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), wave.str_unit, '  |  {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])), spec.str_unit, ' {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])**.5), wave.str_unit)
peak_E = np.ravel(spec.EwEw[:][:]); peak_f = np.nanargmax(peak_E)
print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), wave.str_unit, '  |  {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])), spec.str_unit, ' {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])**.5), wave.str_unit)
if unit_mode==0:
    peak_E = np.ravel(spec.EuEu[:][:]); peak_f = np.nanargmax(peak_E);  print("(RAW)EuEu:", '{:10.0f}'.format(peak_E[peak_f]), '{:x}'.format(np.int64(peak_E[peak_f])))
    peak_E = np.ravel(spec.EvEv[:][:]); peak_f = np.nanargmax(peak_E);  print("(RAW)EvEv:", '{:10.0f}'.format(peak_E[peak_f]), '{:x}'.format(np.int64(peak_E[peak_f])))
    peak_E = np.ravel(spec.EwEw[:][:]); peak_f = np.nanargmax(peak_E);  print("(RAW)EwEw:", '{:10.0f}'.format(peak_E[peak_f]), '{:x}'.format(np.int64(peak_E[peak_f])))

In [ ]:
n_sweep1 = 0;  n_sweep2 = spec.n_time//2;  n_sweep3 = spec.n_time-1
"""
while np.isnan(np.ravel(spec.EuEu[n_sweep1][0])):
    n_sweep1 += 1
while np.isnan(np.ravel(spec.EuEu[n_sweep2][0])):
    n_sweep2 += 1
while np.isnan(np.ravel(spec.EuEu[n_sweep3][0])):
    n_sweep3 -= 1
"""
print(f"[First peak - {n_sweep1}]")
peak_E = np.ravel(spec.EuEu[n_sweep1]); peak_f = np.nanargmax(peak_E); print("EuEu:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EvEv[n_sweep1]); peak_f = np.nanargmax(peak_E); print("EvEv:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EwEw[n_sweep1]); peak_f = np.nanargmax(peak_E); print("EwEw:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
print(f"[Mid   peak - {n_sweep2}]")
peak_E = np.ravel(spec.EuEu[n_sweep2]); peak_f = np.nanargmax(peak_E); print("EuEu:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EvEv[n_sweep2]); peak_f = np.nanargmax(peak_E); print("EvEv:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EwEw[n_sweep2]); peak_f = np.nanargmax(peak_E); print("EwEw:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
print(f"[Last  peak - {n_sweep3}]")
peak_E = np.ravel(spec.EuEu[n_sweep3]); peak_f = np.nanargmax(peak_E); print("EuEu:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EvEv[n_sweep3]); peak_f = np.nanargmax(peak_E); print("EvEv:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EwEw[n_sweep3]); peak_f = np.nanargmax(peak_E); print("EwEw:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))

### Stokes parameters (same with "sample")

In [ ]:
# NAN
if gap_mode == 1 and time_mode == 0:
    for i in range(spec.n_time-1):
        if data.scet[i+1]-data.scet[i] > 60:
            print("[gap]", i, data.epoch[i], i+1, data.epoch[i+1])
            juice_sid2.hf_sid2_spec_nan(data, i);   juice_sid2.hf_sid2_spec_nan(data, i+1)

# Stokes parameters
if comp_mode > 0:                           # [Complex]   0: Poweer  1: Matrix   3: Matrix-2D  >3: any
    spec = juice_spec.hf_getspec_angle (spec, 2)                            # Coherency & ange
    spec = juice_spec.hf_getspec_stokes(spec, 2)                            # Stokes and Polarization Parameters

In [ ]:
n_sweep = spec.n_time//2;  peak_E = np.ravel(spec.EuEu[n_sweep]); peak_f = np.nanargmax(peak_E); 
print(f"[SWEEP - {n_sweep}]  Peak at", '{:.1f}'.format(freq_1d[peak_f]), "kHz  (", peak_f, ")")
print("     EuEu:", '{:+.2e}'.format(np.ravel(spec.EuEu   [n_sweep])[peak_f]), spec.str_unit, "     EvEv:", '{:+.2e}'.format(np.ravel(spec.EvEv   [n_sweep])[peak_f]), spec.str_unit, "   EwEw:", '{:+.2e}'.format(np.ravel(spec.EwEw   [n_sweep])[peak_f]), spec.str_unit)
if comp_mode > 0:                           # [Complex]   0: Poweer  1: Matrix   3: Matrix-2D  >3: any
    print("  EuEv_re:", '{:+.2e}'.format(np.ravel(spec.EuEv_re[n_sweep])[peak_f]), spec.str_unit, "  EuEv_im:", '{:+.2e}'.format(np.ravel(spec.EuEv_im[n_sweep])[peak_f]), spec.str_unit, "  PHAuv:", '{:+5.1f} deg'.format(np.ravel(spec.E_PHAuv[n_sweep])[peak_f]), "  COHuv:", '{:+4.2f}'.format(np.ravel(spec.E_COHuv[n_sweep])[peak_f]), "  DoPuv:", '{:+4.2f}'.format(np.ravel(spec.E_DoPuv[n_sweep])[peak_f]), "  DoLuv:", '{:+4.2f}'.format(np.ravel(spec.E_DoLuv[n_sweep])[peak_f]), "  DoCuv:", '{:+4.2f}'.format(np.ravel(spec.E_DoCuv[n_sweep])[peak_f]), "  ANGuv:", '{:+5.1f} deg'.format(np.ravel(spec.E_ANGuv[n_sweep])[peak_f]))
    print("  EvEw_re:", '{:+.2e}'.format(np.ravel(spec.EvEw_re[n_sweep])[peak_f]), spec.str_unit, "  EvEw_im:", '{:+.2e}'.format(np.ravel(spec.EvEw_im[n_sweep])[peak_f]), spec.str_unit, "  PHAvw:", '{:+5.1f} deg'.format(np.ravel(spec.E_PHAvw[n_sweep])[peak_f]), "  COHvw:", '{:+4.2f}'.format(np.ravel(spec.E_COHvw[n_sweep])[peak_f]), "  DoPvw:", '{:+4.2f}'.format(np.ravel(spec.E_DoPvw[n_sweep])[peak_f]), "  DoLvw:", '{:+4.2f}'.format(np.ravel(spec.E_DoLvw[n_sweep])[peak_f]), "  DoCvw:", '{:+4.2f}'.format(np.ravel(spec.E_DoCvw[n_sweep])[peak_f]), "  ANGvw:", '{:+5.1f} deg'.format(np.ravel(spec.E_ANGvw[n_sweep])[peak_f]))
    print("  EwEu_re:", '{:+.2e}'.format(np.ravel(spec.EwEu_re[n_sweep])[peak_f]), spec.str_unit, "  EwEu_im:", '{:+.2e}'.format(np.ravel(spec.EwEu_im[n_sweep])[peak_f]), spec.str_unit, "  PHAwu:", '{:+5.1f} deg'.format(np.ravel(spec.E_PHAwu[n_sweep])[peak_f]), "  COHqu:", '{:+4.2f}'.format(np.ravel(spec.E_COHwu[n_sweep])[peak_f]), "  DoPwu:", '{:+4.2f}'.format(np.ravel(spec.E_DoPwu[n_sweep])[peak_f]), "  DoLwu:", '{:+4.2f}'.format(np.ravel(spec.E_DoLwu[n_sweep])[peak_f]), "  DoCwu:", '{:+4.2f}'.format(np.ravel(spec.E_DoCwu[n_sweep])[peak_f]), "  ANGwu:", '{:+5.1f} deg'.format(np.ravel(spec.E_ANGwu[n_sweep])[peak_f]))
    print('Stokes-UV: {:+.2e}'.format(np.ravel(spec.E_Iuv[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Quv[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Uuv[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Vuv[n_sweep])[peak_f]))
    print('Stokes-VW: {:+.2e}'.format(np.ravel(spec.E_Ivw[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Qvw[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Uvw[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Vvw[n_sweep])[peak_f]))
    print('Stokes-WU: {:+.2e}'.format(np.ravel(spec.E_Iwu[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Qwu[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Uwu[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Vwu[n_sweep])[peak_f]))

## Spectrum plot (same with "sample")

In [ ]:
EE_2d      = spec.EE.transpose();       EE_med      = np.nanmedian(spec.EE,      axis=0)
EuEu_2d    = spec.EuEu.transpose();     EuEu_med    = np.nanmedian(spec.EuEu,    axis=0)
EvEv_2d    = spec.EvEv.transpose();     EvEv_med    = np.nanmedian(spec.EvEv,    axis=0)
EwEw_2d    = spec.EwEw.transpose();     EwEw_med    = np.nanmedian(spec.EwEw,    axis=0)
if comp_mode > 0:                           # [Complex]   0: Poweer  1: Matrix   3: Matrix-2D  >3: any
    E_DoPuv_2d = spec.E_DoPuv.transpose();  E_DoPuv_med = np.nanmedian(spec.E_DoPuv, axis=0)
    E_DoPvw_2d = spec.E_DoPvw.transpose();  E_DoPvw_med = np.nanmedian(spec.E_DoPvw, axis=0)
    E_DoPwu_2d = spec.E_DoPwu.transpose();  E_DoPwu_med = np.nanmedian(spec.E_DoPwu, axis=0)
    E_DoLuv_2d = spec.E_DoLuv.transpose();  E_DoLuv_med = np.nanmedian(spec.E_DoLuv, axis=0)
    E_DoLvw_2d = spec.E_DoLvw.transpose();  E_DoLvw_med = np.nanmedian(spec.E_DoLvw, axis=0)
    E_DoLwu_2d = spec.E_DoLwu.transpose();  E_DoLwu_med = np.nanmedian(spec.E_DoLwu, axis=0)
    E_DoCuv_2d = spec.E_DoCuv.transpose();  E_DoCuv_med = np.nanmedian(spec.E_DoCuv, axis=0)
    E_DoCvw_2d = spec.E_DoCvw.transpose();  E_DoCvw_med = np.nanmedian(spec.E_DoCvw, axis=0)
    E_DoCwu_2d = spec.E_DoCwu.transpose();  E_DoCwu_med = np.nanmedian(spec.E_DoCwu, axis=0)
    E_ANGuv_2d = spec.E_ANGuv.transpose();  E_ANGuv_med = np.nanmedian(spec.E_ANGuv, axis=0)
    E_ANGvw_2d = spec.E_ANGvw.transpose();  E_ANGvw_med = np.nanmedian(spec.E_ANGvw, axis=0)
    E_ANGwu_2d = spec.E_ANGwu.transpose();  E_ANGwu_med = np.nanmedian(spec.E_ANGwu, axis=0)
    E_COHuv_2d = spec.E_COHuv.transpose();  E_COHuv_med = np.nanmedian(spec.E_COHuv, axis=0)
    E_COHvw_2d = spec.E_COHvw.transpose();  E_COHvw_med = np.nanmedian(spec.E_COHvw, axis=0)
    E_COHwu_2d = spec.E_COHwu.transpose();  E_COHwu_med = np.nanmedian(spec.E_COHwu, axis=0)
    E_PHAuv_2d = spec.E_PHAuv.transpose();  E_PHAuv_med = np.nanmedian(spec.E_PHAuv, axis=0)
    E_PHAvw_2d = spec.E_PHAvw.transpose();  E_PHAvw_med = np.nanmedian(spec.E_PHAvw, axis=0)
    E_PHAwu_2d = spec.E_PHAwu.transpose();  E_PHAwu_med = np.nanmedian(spec.E_PHAwu, axis=0)

In [ ]:
n_sweep1 = 0;  n_sweep2 = spec.n_time//2;  n_sweep3 = spec.n_time-1
p_min = p_min1;  p_max = p_max1
f_min = f_min1;  f_max = f_max1
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max

fig = plt.figure(figsize=(16, 11))
ax1 = fig.add_subplot(3, 1, 1);  ax2 = fig.add_subplot(3, 1, 2);  ax3 = fig.add_subplot(3, 1, 3)

ax1.plot(freq_1d, spec.EuEu[n_sweep1], '-r', linewidth=.9, label='EuEu 1st')
ax1.plot(freq_1d, spec.EuEu[n_sweep2], '-g', linewidth=.4, label='EuEu mid')
ax1.plot(freq_1d, spec.EuEu[n_sweep3], '-b', linewidth=.2, label='EuEu last')
ax2.plot(freq_1d, spec.EvEv[n_sweep1], '-r', linewidth=.9, label='EvEv 1st')
ax2.plot(freq_1d, spec.EvEv[n_sweep2], '-g', linewidth=.4, label='EvEv mid')
ax2.plot(freq_1d, spec.EvEv[n_sweep3], '-b', linewidth=.2, label='EvEv last')
ax3.plot(freq_1d, spec.EwEw[n_sweep1], '-r', linewidth=.9, label='EwEw 1st')
ax3.plot(freq_1d, spec.EwEw[n_sweep2], '-g', linewidth=.4, label='EwEw mid')
ax3.plot(freq_1d, spec.EwEw[n_sweep3], '-b', linewidth=.2, label='EwEw last')
ax1.set_yscale('log');      ax2.set_yscale('log');  ax3.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log');  ax2.set_xscale('log');  ax3.set_xscale('log')
ax1.set_ylabel('EuEu '+spec.str_unit);     ax2.set_ylabel('EvEv '+spec.str_unit); ax3.set_ylabel('EwEw '+spec.str_unit);  ax3.set_xlabel('Frequency [kHz]')

date1 = spec.epoch[n_sweep1];  date1 = date1.strftime('1st: %Y/%m/%d %R:%S   ')
date2 = spec.epoch[n_sweep2];  date2 = date2.strftime('Mid: %Y/%m/%d %R:%S   ')
date3 = spec.epoch[n_sweep3];  date3 = date3.strftime('Fin: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3;   ax1.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)

xlim=[f_min, f_max]
ax1.set_xlim(xlim); ax2.set_xlim(xlim);  ax3.set_xlim(xlim); ax4.set_xlim(xlim)
ylim=[10**(p_min), 10**(p_max)]
ax1.set_ylim(ylim); ax2.set_ylim(ylim);  ax3.set_ylim(ylim); ax4.set_ylim(ylim)

fig.subplots_adjust(hspace=0)
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec-log.png'
    fig.savefig(png_fname)

In [ ]:
n_sweep1 = 0;  n_sweep2 = spec.n_time//2;  n_sweep3 = spec.n_time-1
p_min = p_min1;  p_max = p_max1
f_min = f_min1;  f_max = f_max1

fig = plt.figure(figsize=(16, 11));  ax1 = fig.add_subplot(1, 1, 1)
# ax1.plot(freq_1d, EE_med  ,      '-y',  linewidth=4.0, label='EE_med')
ax1.plot(freq_1d, EuEu_med,      '-r',  linewidth=0.9, label='EuEu_med')
ax1.plot(freq_1d, EvEv_med,      '-g',  linewidth=0.4, label='EvEv_med')
ax1.plot(freq_1d, EwEw_med,      '-b',  linewidth=0.2, label='EwEw_med')
"""
ax1.plot(freq_1d, spec.EuEu[n_sweep1], ':r',  linewidth=0.9, label='uu init')
ax1.plot(freq_1d, spec.EvEv[n_sweep1], ':g',  linewidth=0.4, label='vv init')
ax1.plot(freq_1d, spec.EwEw[n_sweep1], ':b',  linewidth=0.2, label='ww init')
ax1.plot(freq_1d, spec.EuEu[n_sweep2], '-.r', linewidth=0.9, label='uu half')
ax1.plot(freq_1d, spec.EvEv[n_sweep2], '-.g', linewidth=0.4, label='vv half')
ax1.plot(freq_1d, spec.EwEw[n_sweep2], '-.b', linewidth=0.2, label='ww half')
ax1.plot(freq_1d, spec.EuEu[n_sweep3], '-r',  linewidth=0.9, label='uu end')
ax1.plot(freq_1d, spec.EvEv[n_sweep3], '-g',  linewidth=0.4, label='vv end')
ax1.plot(freq_1d, spec.EwEw[n_sweep3], '-b',  linewidth=0.2, label='ww end')
"""
ax1.set_yscale('log');      ax2.set_yscale('log');  ax3.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log');  ax2.set_xscale('log');  ax3.set_xscale('log')
ax1.set_ylabel('EuEu '+spec.str_unit);     ax2.set_ylabel('EvEv '+spec.str_unit); ax3.set_ylabel('EwEw '+spec.str_unit);  ax3.set_xlabel('Frequency [kHz]')

date1 = spec.epoch[n_sweep1];  date1 = date1.strftime('1st: %Y/%m/%d %R:%S   ')
date2 = spec.epoch[n_sweep2];  date2 = date2.strftime('Mid: %Y/%m/%d %R:%S   ')
date3 = spec.epoch[n_sweep3];  date3 = date3.strftime('Fin: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3;   ax1.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)

xlim=[f_min, f_max]
ax1.set_xlim(xlim)
ylim=[10**p_min, 10**p_max]; 
ax1.set_ylim(ylim)

fig.subplots_adjust(hspace=0)
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec2.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec2-log.png'
    fig.savefig(png_fname)

In [ ]:
n_sweep1 = 0;  n_sweep2 = spec.n_time//2;  n_sweep3 = spec.n_time-1
p_min = p_min1;  p_max = p_max1
f_min = 0;       f_max = 5000

fig = plt.figure(figsize=(16, 11))
ax1 = fig.add_subplot(9, 1, 1);  ax2 = fig.add_subplot(9, 1, 2);  ax3 = fig.add_subplot(9, 1, 3);  ax4 = fig.add_subplot(9, 1, 4)
ax5 = fig.add_subplot(9, 1, 5);  ax6 = fig.add_subplot(9, 1, 6);  ax7 = fig.add_subplot(9, 1, 7);  ax8 = fig.add_subplot(9, 1, 8);  ax9 = fig.add_subplot(9, 1, 9)

ax1.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax1.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE mid')
ax1.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax1.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax2.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax2.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE mid')
ax2.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax2.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax3.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax3.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE mid')
ax3.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax3.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax4.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax4.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE mid')
ax4.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax4.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax5.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax5.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE mid')
ax5.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax5.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax6.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax6.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE mid')
ax6.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax6.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax7.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax7.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE mid')
ax7.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax7.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax8.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax8.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE mid')
ax8.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax8.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax9.plot(freq_1d, spec.EE[n_sweep1], ':r', linewidth=.7, label='EE 1st'); ax9.plot(freq_1d, spec.EE[n_sweep2], ':g', linewidth=.7, label='EE mid')
ax9.plot(freq_1d, spec.EE[n_sweep3], ':b', linewidth=.7, label='EE fin'); ax9.plot(freq_1d, EE_med,            '-k', linewidth=.7, label='EE ave')
ax1.set_yscale('log');  ax2.set_yscale('log');  ax3.set_yscale('log');  ax4.set_yscale('log');  ax5.set_yscale('log')
ax6.set_yscale('log');  ax7.set_yscale('log');  ax8.set_yscale('log');  ax9.set_yscale('log')

ax1.set_ylabel(spec.str_unit); ax2.set_ylabel(spec.str_unit); ax3.set_ylabel(spec.str_unit); ax4.set_ylabel(spec.str_unit); ax5.set_ylabel(spec.str_unit)
ax6.set_ylabel(spec.str_unit); ax7.set_ylabel(spec.str_unit); ax8.set_ylabel(spec.str_unit); ax9.set_ylabel(spec.str_unit)
ax9.set_xlabel('Frequency [kHz]')
date1 = spec.epoch[n_sweep1];  date1 = date1.strftime(f'1st {n_sweep1}: %Y/%m/%d %R:%S ')
date2 = spec.epoch[n_sweep2];  date2 = date2.strftime(f'Mid {n_sweep2}: %Y/%m/%d %R:%S ')
date3 = spec.epoch[n_sweep3];  date3 = date3.strftime(f'Fin {n_sweep3}: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3;  ax1.set_title(title_date)

ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8);  ax6.legend(loc='upper right', fontsize=8)
ax7.legend(loc='upper right', fontsize=8);  ax8.legend(loc='upper right', fontsize=8);  ax9.legend(loc='upper right', fontsize=8)

xlim=[f_min,       f_max];        ax1.set_xlim(xlim); xlim=[f_min+5000,  f_max+5000];   ax2.set_xlim(xlim)
xlim=[f_min+10000, f_max+10000];  ax3.set_xlim(xlim); xlim=[f_min+15000, f_max+15000];  ax4.set_xlim(xlim)
xlim=[f_min+20000, f_max+20000];  ax5.set_xlim(xlim); xlim=[f_min+25000, f_max+25000];  ax6.set_xlim(xlim)
xlim=[f_min+30000, f_max+30000];  ax7.set_xlim(xlim); xlim=[f_min+35000, f_max+35000];  ax8.set_xlim(xlim)
xlim=[f_min+40000, f_max+40000];  ax9.set_xlim(xlim)
ax1.tick_params(labelsize=7); ax2.tick_params(labelsize=7); ax3.tick_params(labelsize=7); ax4.tick_params(labelsize=7); ax5.tick_params(labelsize=7)
ax6.tick_params(labelsize=7); ax7.tick_params(labelsize=7); ax8.tick_params(labelsize=7); ax9.tick_params(labelsize=7)

ylim=[10**p_min, 10**p_max]
ax1.set_ylim(ylim); ax2.set_ylim(ylim); ax3.set_ylim(ylim); ax4.set_ylim(ylim); ax5.set_ylim(ylim); ax6.set_ylim(ylim);  
ax7.set_ylim(ylim); ax8.set_ylim(ylim); ax9.set_ylim(ylim)

fig.subplots_adjust(hspace=0.2)
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-fine.png'
    fig.savefig(png_fname)

In [ ]:
p_min = p_min1;  p_max = p_max1
f_min = f_min1;  f_max = f_max1
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max;  print(data.frequency[0][0][0], data.frequency[0][8][0])

fig2d = plt.figure(figsize=[16,11])
if time_mode == 1:
    ax1 = fig2d.add_subplot(4, 1, 1); ax2 = fig2d.add_subplot(4, 1, 2); ax3 = fig2d.add_subplot(4, 1, 3); ax4.set_xlabel(str_date); ax4 = fig2d.add_subplot(4, 1, 4)
else:
    ax1 = fig2d.add_subplot(3, 1, 1); ax2 = fig2d.add_subplot(3, 1, 2); ax3 = fig2d.add_subplot(3, 1, 3); ax3.set_xlabel(str_date)

ax1.set_ylabel('EuEu [kHz]'); ax2.set_ylabel('EvEv [kHz]'); ax3.set_ylabel('EwEw [kHz]')
ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log');    ax2.set_yscale('log');        ax3.set_yscale('log')

if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d, EuEu_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(num_1d, freq_1d, EvEv_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(num_1d, freq_1d, EwEw_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p4 = ax4.plot(np.ravel(spec.epoch[:]), '.')
    pp3 = fig2d.colorbar(p3, ax=ax4, orientation="vertical"); pp3.set_label(spec.str_unit)
else:
    p1 = ax1.pcolormesh(spec.epoch, freq_1d, EuEu_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(spec.epoch, freq_1d, EvEv_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(spec.epoch, freq_1d, EwEw_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp1.set_label(spec.str_unit)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical"); pp2.set_label(spec.str_unit)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp3.set_label(spec.str_unit)

print( "(", num_1d[0],")", spec.epoch[0], "-", "(", num_1d[-1],")", spec.epoch[-1] )
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]; ax4.set_xlim(xlim) 
else:
    if t_min0 == 0:
        xlim=[spec.epoch[0], spec.epoch[-1]]
    else:
        xlim=[t_min0, t_max0]
    # E_min = '2024-08-20 18:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
    # E_max = '2024-08-21 05:30:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
    print("==>", xlim)
ax1.set_xlim(xlim);  ax2.set_xlim(xlim); ax3.set_xlim(xlim)

plt.subplots_adjust(hspace=.03)
plt.show
if dump_mode > 0:
    png_fname = work_dir+data_name+'_FT.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-log.png'
    fig2d.savefig(png_fname)

# Mask Write / Read for SID-3

In [ ]:
freq_1d_sid3, f_step_sid3, f_width_sid3 = juice_cdf._frequency_sid3(asw_ver)
n_freq_sid3 = len(freq_1d_sid3)

print( "  sid:", asw_ver)
print( "SID-2:", freq_1d.shape[0],        " freq:", freq_1d[0],      "-", freq_1d[-1],      "[kHz]", "  step", freq_w_1d[0],   freq_w_1d[-1],   "(", freq_1d[0]     - freq_w_1d[0]/2,   freq_1d[-1] - freq_w_1d[-1]/2,      ")", )
print( "SID-3:", freq_1d_sid3.shape[0], "   freq:", freq_1d_sid3[0], "-", freq_1d_sid3[-1], "[kHz]", "  step", f_step_sid3[0], f_step_sid3[-1], "(", freq_1d_sid3[0]- f_step_sid3[0]/2, freq_1d_sid3[-1]-f_step_sid3[-1]/2, ")", )

In [ ]:
freq_1d_msk = np.zeros(n_freq_msk); freq_w_msk = np.zeros(n_freq_msk);  EE_msk = np.zeros(n_freq_msk)
for i in range(n_freq_msk):
    freq_1d_msk[i] = f_min_msk + f_step_msk * (i + 0.5)      # Central frequency of each channel
freq_w_msk[:] = f_step_msk
print( " mask:", freq_1d_msk.shape[0], " freq:", freq_1d_msk[0], "-", freq_1d_msk[-1], "[kHz]", "  step", freq_w_msk[0], freq_w_msk[-1], "(", freq_1d_msk[0] - freq_w_msk[0]/2, freq_1d_msk[-1] - freq_w_msk[-1]/2, ")", )
print( "SID-2:", freq_1d.shape[0],     " freq:", freq_1d[0],     "-", freq_1d[-1],     "[kHz]", "  step", freq_w_1d[0],  freq_w_1d[-1],  "(", freq_1d[0]     - freq_w_1d[0]/2,  freq_1d[-1]     - freq_w_1d[-1]/2,  ")", )

# SID2 != MASK table
j0 = 0
for i in range(n_freq_msk):
    if freq_1d_msk[i] > freq_1d[0]-freq_w_1d[0]/2 and freq_1d_msk[i] < freq_1d[-1]+freq_w_1d[-1]/2:
        for j in range(j0, n_freq_msk):
            if freq_1d_msk[i] >= freq_1d[j]-freq_w_1d[j]/2:
                if freq_1d_msk[i] < freq_1d[j]+freq_w_1d[j]/2:
                    EE_msk[i] = EE_med[j]
                    break
        j0 = j+1
        if j0 >= n_freq_msk:  break
    else:
        EE_msk[i] = EE_med[i]

In [ ]:
sid3_EE_med = np.zeros(n_freq_sid3); sid3_EE_ave = np.zeros(n_freq_sid3); sid3_EE_msk = np.zeros(n_freq_sid3)   # median / averaged / masked
EE_med_sid3 = np.zeros(n_freq1);     EE_ave_sid3 = np.zeros(n_freq1);     EE_msk_sid3 = np.zeros(n_freq1)       # median / averaged / masked
EE_diff     = np.zeros(n_freq1)      # difference

# SID-3 Frequency from MASK     --  [tbl_freq_to_sid]  0 - start ch, 1 - end ch,  2 - width ch
tbl_sid3_to_sid2 = np.zeros((n_freq1, 4), dtype = float)
tbl_msk_to_sid3  = juice_cdf._frequency_sid2_to_data(freq_1d_sid3, f_step_sid3, freq_1d_msk, freq_w_msk)
tbl = tbl_msk_to_sid3[0];  EE_med_sid3[0:tbl[0]] = math.nan;  EE_ave_sid3[0:tbl[0]] = math.nan;  EE_msk_sid3[0:tbl[0]] = math.nan
for j in range(n_freq_sid3):
    tbl = tbl_msk_to_sid3[j]                        # 0 - start ch, 1 - end ch, 2 - width ch
    sid3_EE_med[j]  = 10 ** np.median( np.log10(EE_msk[tbl[0]:tbl[1]+1]) )  
    sid3_EE_ave[j]  = np.mean( EE_msk[tbl[0]:tbl[1]+1] )    
    EE_med_sid3     [tbl[0]:tbl[1]+1]      = sid3_EE_med[j]
    EE_ave_sid3     [tbl[0]:tbl[1]+1]      = sid3_EE_ave[j]
    tbl_sid3_to_sid2[tbl[0]:tbl[1]+1, 0:3] = tbl    # 0 - start ch, 1 - end ch, 2 - width ch
    tbl_sid3_to_sid2[tbl[0]:tbl[1]+1, 3]   = j      # 3 - SID-3 channel
EE_med_sid3[tbl[1]:n_freq1] = math.nan;  EE_ave_sid3[tbl[1]:n_freq1] = math.nan;  EE_msk_sid3[tbl[1]:n_freq1] = math.nan
for i in range(n_freq_msk):
    EE_diff[i] = 10 * np.log10(EE_msk[i]) - 10 * np.log10(EE_med_sid3[i] * 10**(sid3_diff/10.))

In [ ]:
p_min = p_min1;  p_max = p_max1
f_min = f_min1;  f_max = f_max1
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max

fig = plt.figure(figsize=(14, 11));  ax1 = fig.add_subplot(1, 1, 1)
ax1.plot(freq_1d,      EE_med,      '-y', linewidth=4,  label='EE_org   (SID2)')
ax1.plot(freq_1d_msk,  EE_msk,      '-b', linewidth=.5, label='EE_masked(SID2)')
ax1.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log')
xlim=[f_min, f_max];                ax1.set_xlim(xlim)
ylim=[10**(p_min), 10**(p_max)];    ax1.set_ylim(ylim)

ax1.set_ylabel('EE '+spec.str_unit)
date1 = spec.epoch[n_sweep1];  date1 = date1.strftime('1st: %Y/%m/%d %R:%S   ')
date2 = spec.epoch[n_sweep2];  date2 = date2.strftime('Mid: %Y/%m/%d %R:%S   ')
date3 = spec.epoch[n_sweep3];  date3 = date3.strftime('Fin: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3
ax1.set_title(title_date);   ax1.legend(loc='upper right', fontsize=8)
fig.show

In [ ]:
# init: MASK
mask_list = np.zeros((n_freq_msk, 4), dtype = float)
mask_dir  = './lib/';  mask_name = 'hf_sid3_mask.csv';  mask_file = mask_dir + mask_name

# Write Mask
if filter_mode > 0:
    for i in range(n_freq_msk):
        mask_list[i, 0] = freq_1d_msk[i] - 2.3125/2     # f_min
        mask_list[i, 1] = freq_1d_msk[i] + 2.3125/2     # f_max
        mask_list[i, 2] = 0                             # mask
        if EE_diff[i] >= 0: mask_list[i, 2] = 1
    for i in range(n_freq_sid3):
        num_mask = 0
        tbl = tbl_msk_to_sid3[i]                        # 0 - start ch, 1 - end ch, 2 - width ch
        for j in range (tbl[0], tbl[1]+1):
            if mask_list[j, 2] == 0:  num_mask += 1.0
        for j in range (tbl[0], tbl[1]+1):
            mask_list[j, 3] = num_mask / tbl[2]
    with open(mask_file, 'w') as f:
        writer = csv.writer(f)
        for i in range(n_freq_msk):
            if filter_mode > 1:
                writer.writerow([ i+1, mask_list[i, 0], mask_list[i, 1], np.int16(mask_list[i, 2]), mask_list[i, 3], 10 * np.log10(EE_msk[i]), 10 * np.log10(EE_med_sid3[i]), EE_diff[i], tbl_sid3_to_sid2[i, 0], tbl_sid3_to_sid2[i, 1], tbl_sid3_to_sid2[i, 2], tbl_sid3_to_sid2[i, 3]])
            else:
                writer.writerow([ i+1, mask_list[i, 0], mask_list[i, 1], np.int16(mask_list[i, 2])])

# Read Mask
with open(mask_file, 'r') as f:
    reader = csv.reader(f);  i = 0
    for row in reader:
        mask_list[i, 0] = row[1];  mask_list[i, 1] = row[2];  mask_list[i, 2] = row[3];  i = i+1
print("mask read: ", mask_list.shape, mask_file)

In [ ]:
EE_msk2 = copy.deepcopy(EE_msk)
index   = np.where(mask_list[:, 2] > 0);  EE_msk2[index[0]] = 0.0

# SID-3 from masked SID-2 
for j in range(n_freq_sid3):
    j0 = tbl_msk_to_sid3[j][0];  j1 = tbl_msk_to_sid3[j][1]
    num            = np.sum(1-mask_list[j0:j1+1, 2])
    sid3_EE_msk[j] = np.sum(EE_msk2[j0:j1+1]) / num
    EE_msk_sid3[j0:j1+1] = sid3_EE_msk[j]

### SID-3 Frequency table with MASK

In [ ]:
# SID-3 Frequeny table write ---- ch, frequency, num of SID-2 ch, masked ch, masked ratio, f_min, f_max
if filter_sid3>0:
    freq_dir  = './lib/';  freq_name = 'hf_sid3_freq.csv';  freq_file = freq_dir + freq_name
    with open(freq_file, 'w') as f:
        writer = csv.writer(f)
        for i in range(n_freq_sid3):
            i0 = tbl_msk_to_sid3[i][0];  i1 = tbl_msk_to_sid3[i][1]
            num = 0
            for j in range(i0, i1+1):
                if EE_msk2[j]==0:
                    num = num +1
            writer.writerow([ i+1, freq_1d_sid3[i], tbl_msk_to_sid3[i][2], num, num/tbl_msk_to_sid3[i][2]*100, freq_1d_msk[i0]-freq_w_msk[i0]/2, freq_1d_msk[i1]+freq_w_msk[i1]/2 ])

# SID3 spectrum with Mask

In [ ]:
p_min = p_min1; p_max = p_max1
f_min = f_min1; f_max = f_max1
# f_min = 150;    f_max = 1050
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max

fig = plt.figure(figsize=(14, 11));  ax1 = fig.add_subplot(1, 1, 1)
ax1.plot(freq_1d_msk,  EE_msk,      '-y', linewidth=.5, label='EE_org (SID2)')
ax1.plot(freq_1d_msk,  EE_msk2,      '.r', linewidth=.5, label='EE_masked(SID2)')
ax1.plot(freq_1d_msk,  EE_msk_sid3, '-c', linewidth=4,  label='EE_msk (SID2)')
ax1.plot(freq_1d_msk,  EE_ave_sid3, '-r', linewidth=1,  label='EE_ave (SID2)')
ax1.plot(freq_1d_msk,  EE_med_sid3, '-b', linewidth=1,  label='EE_med (SID2)')
#ax1.plot(freq_1d_sid3, sid3_EE_ave, '-r', linewidth=1,  label='EE_ave (SID3)')
#ax1.plot(freq_1d_sid3, sid3_EE_med, '-c', linewidth=3,  label='EE_med (SID3)')
#ax1.plot(freq_1d_sid3, sid3_EE_msk, '-b', linewidth=1,  label='EE_msk (SID3)')
ax1.plot(freq_1d_msk,  mask_list[:, 2]*1E6,  '.k', linewidth=.3,  label='mask pattern')
ax1.plot(freq_1d_msk,  10**mask_list[:, 3],   '-k', linewidth=.3,  label='mask ratio')
ax1.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log');      ax2.set_xscale('log')
xlim=[f_min, f_max];              ax1.set_xlim(xlim)
ylim=[10**(p_min), 10**(p_max)];  ax1.set_ylim(ylim)

ax1.set_ylabel('EE '+spec.str_unit)
date1 = spec.epoch[n_sweep1];  date1 = date1.strftime('1st: %Y/%m/%d %R:%S   ')
date2 = spec.epoch[n_sweep2];  date2 = date2.strftime('Mid: %Y/%m/%d %R:%S   ')
date3 = spec.epoch[n_sweep3];  date3 = date3.strftime('Fin: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3
ax1.set_title(title_date);   ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  

fig.show

In [ ]:
p_min = p_min1;   p_max = p_max1
f_min = 0;        f_max = 5000

fig = plt.figure(figsize=(14, 11))
ax1 = fig.add_subplot(9, 1, 1);  ax2 = fig.add_subplot(9, 1, 2);  ax3 = fig.add_subplot(9, 1, 3);  ax4 = fig.add_subplot(9, 1, 4)
ax5 = fig.add_subplot(9, 1, 5);  ax6 = fig.add_subplot(9, 1, 6);  ax7 = fig.add_subplot(9, 1, 7);  ax8 = fig.add_subplot(9, 1, 8)
ax9 = fig.add_subplot(9, 1, 9)

ax1.plot(freq_1d_msk,  EE_msk,      '.y', linewidth=.5, label='EE_org (SID2)')
ax1.plot(freq_1d_msk,  EE_ave_sid3, '-r', linewidth=1,  label='EE_ave (SID2)')
ax1.plot(freq_1d_msk,  EE_med_sid3, '-c', linewidth=3,  label='EE_med (SID2)')
ax1.plot(freq_1d_msk,  EE_msk_sid3, '-b', linewidth=1,  label='EE_msk (SID2)')
#ax1.plot(freq_1d_sid3, sid3_EE_ave, '-r', linewidth=1,  label='EE_ave (SID3)')
#ax1.plot(freq_1d_sid3, sid3_EE_med, '-c', linewidth=3,  label='EE_med (SID3)')
#ax1.plot(freq_1d_sid3, sid3_EE_msk, '-b', linewidth=1,  label='EE_msk (SID3)')
ax1.plot(freq_1d_msk,  mask_list[:, 2]*1E6,   '.k', linewidth=.3,  label='mask pattern')
#
ax2.plot(freq_1d_msk,  EE_msk,      '.y', linewidth=.5, label='EE_org (SID2)')
ax2.plot(freq_1d_msk,  EE_ave_sid3, '-r', linewidth=1,  label='EE_ave (SID2)')
ax2.plot(freq_1d_msk,  EE_med_sid3, '-c', linewidth=3,  label='EE_med (SID2)')
ax2.plot(freq_1d_msk,  EE_msk_sid3, '-b', linewidth=1,  label='EE_msk (SID2)')
#ax2.plot(freq_1d_sid3, sid3_EE_ave, '-r', linewidth=1,  label='EE_ave (SID3)')
#ax2.plot(freq_1d_sid3, sid3_EE_med, '-c', linewidth=3,  label='EE_med (SID3)')
#ax2.plot(freq_1d_sid3, sid3_EE_msk, '-b', linewidth=1,  label='EE_msk (SID3)')
ax2.plot(freq_1d_msk,  mask_list[:, 2]*1E6,   '.k', linewidth=.3,  label='mask pattern')
#
ax3.plot(freq_1d_msk,  EE_msk,      '.y', linewidth=.5, label='EE_org (SID2)')
ax3.plot(freq_1d_msk,  EE_ave_sid3, '-r', linewidth=1,  label='EE_ave (SID2)')
ax3.plot(freq_1d_msk,  EE_med_sid3, '-c', linewidth=3,  label='EE_med (SID2)')
ax3.plot(freq_1d_msk,  EE_msk_sid3, '-b', linewidth=1,  label='EE_msk (SID2)')
#ax3.plot(freq_1d_sid3, sid3_EE_ave, '-r', linewidth=1,  label='EE_ave (SID3)')
#ax3.plot(freq_1d_sid3, sid3_EE_med, '-c', linewidth=3,  label='EE_med (SID3)')
#ax3.plot(freq_1d_sid3, sid3_EE_msk, '-b', linewidth=1,  label='EE_msk (SID3)')
ax3.plot(freq_1d_msk,  mask_list[:, 2]*1E6,   '.k', linewidth=.3,  label='mask pattern')
#
ax4.plot(freq_1d_msk,  EE_msk,      '.y', linewidth=.5, label='EE_org (SID2)')
ax4.plot(freq_1d_msk,  EE_ave_sid3, '-r', linewidth=1,  label='EE_ave (SID2)')
ax4.plot(freq_1d_msk,  EE_med_sid3, '-c', linewidth=3,  label='EE_med (SID2)')
ax4.plot(freq_1d_msk,  EE_msk_sid3, '-b', linewidth=1,  label='EE_msk (SID2)')
#ax4.plot(freq_1d_sid3, sid3_EE_ave, '-r', linewidth=1,  label='EE_ave (SID3)')
#ax4.plot(freq_1d_sid3, sid3_EE_med, '-c', linewidth=3,  label='EE_med (SID3)')
#ax4.plot(freq_1d_sid3, sid3_EE_msk, '-b', linewidth=1,  label='EE_msk (SID3)')
ax4.plot(freq_1d_msk,  mask_list[:, 2]*1E6,   '.k', linewidth=.3,  label='mask pattern')
#
ax5.plot(freq_1d_msk,  EE_msk,      '.y', linewidth=.5, label='EE_org (SID2)')
ax5.plot(freq_1d_msk,  EE_ave_sid3, '-r', linewidth=1,  label='EE_ave (SID2)')
ax5.plot(freq_1d_msk,  EE_med_sid3, '-c', linewidth=3,  label='EE_med (SID2)')
ax5.plot(freq_1d_msk,  EE_msk_sid3, '-b', linewidth=1,  label='EE_msk (SID2)')
#ax5.plot(freq_1d_sid3, sid3_EE_ave, '-r', linewidth=1,  label='EE_ave (SID3)')
#ax5.plot(freq_1d_sid3, sid3_EE_med, '-c', linewidth=3,  label='EE_med (SID3)')
#ax5.plot(freq_1d_sid3, sid3_EE_msk, '-b', linewidth=1,  label='EE_msk (SID3)')
ax5.plot(freq_1d_msk,  mask_list[:, 2]*1E6,   '.k', linewidth=.3,  label='mask pattern')
#
ax6.plot(freq_1d_msk,  EE_msk,      '.y', linewidth=.5, label='EE_org (SID2)')
ax6.plot(freq_1d_msk,  EE_ave_sid3, '-r', linewidth=1,  label='EE_ave (SID2)')
ax6.plot(freq_1d_msk,  EE_med_sid3, '-c', linewidth=3,  label='EE_med (SID2)')
ax6.plot(freq_1d_msk,  EE_msk_sid3, '-b', linewidth=1,  label='EE_msk (SID2)')
#ax6.plot(freq_1d_sid3, sid3_EE_ave, '-r', linewidth=1,  label='EE_ave (SID3)')
#ax6.plot(freq_1d_sid3, sid3_EE_med, '-c', linewidth=3,  label='EE_med (SID3)')
#ax6.plot(freq_1d_sid3, sid3_EE_msk, '-b', linewidth=1,  label='EE_msk (SID3)')
ax6.plot(freq_1d_msk,  mask_list[:, 2]*1E6,   '.k', linewidth=.3,  label='mask pattern')
#
ax7.plot(freq_1d_msk,  EE_msk,      '.y', linewidth=.5, label='EE_org (SID2)')
ax7.plot(freq_1d_msk,  EE_ave_sid3, '-r', linewidth=1,  label='EE_ave (SID2)')
ax7.plot(freq_1d_msk,  EE_med_sid3, '-c', linewidth=3,  label='EE_med (SID2)')
ax7.plot(freq_1d_msk,  EE_msk_sid3, '-b', linewidth=1,  label='EE_msk (SID2)')
#ax7.plot(freq_1d_sid3, sid3_EE_ave, '-r', linewidth=1,  label='EE_ave (SID3)')
#ax7.plot(freq_1d_sid3, sid3_EE_med, '-c', linewidth=3,  label='EE_med (SID3)')
#ax7.plot(freq_1d_sid3, sid3_EE_msk, '-b', linewidth=1,  label='EE_msk (SID3)')
ax7.plot(freq_1d_msk,  mask_list[:, 2]*1E6,   '.k', linewidth=.3,  label='mask pattern')
#
ax8.plot(freq_1d_msk,  EE_msk,      '.y', linewidth=.5, label='EE_org (SID2)')
ax8.plot(freq_1d_msk,  EE_ave_sid3, '-r', linewidth=1,  label='EE_ave (SID2)')
ax8.plot(freq_1d_msk,  EE_med_sid3, '-c', linewidth=3,  label='EE_med (SID2)')
ax8.plot(freq_1d_msk,  EE_msk_sid3, '-b', linewidth=1,  label='EE_msk (SID2)')
#ax8.plot(freq_1d_sid3, sid3_EE_ave, '-r', linewidth=1,  label='EE_ave (SID3)')
#ax8.plot(freq_1d_sid3, sid3_EE_med, '-c', linewidth=3,  label='EE_med (SID3)')
#ax8.plot(freq_1d_sid3, sid3_EE_msk, '-b', linewidth=1,  label='EE_msk (SID3)')
ax8.plot(freq_1d_msk,  mask_list[:, 2]*1E6,   '.k', linewidth=.3,  label='mask pattern')
#
ax9.plot(freq_1d_msk,  EE_msk,      '.y', linewidth=.5, label='EE_org (SID2)')
ax9.plot(freq_1d_msk,  EE_ave_sid3, '-r', linewidth=1,  label='EE_ave (SID2)')
ax9.plot(freq_1d_msk,  EE_med_sid3, '-c', linewidth=3,  label='EE_med (SID2)')
ax9.plot(freq_1d_msk,  EE_msk_sid3, '-b', linewidth=1,  label='EE_msk (SID2)')
#ax9.plot(freq_1d_sid3, sid3_EE_ave, '-r', linewidth=1,  label='EE_ave (SID3)')
#ax9.plot(freq_1d_sid3, sid3_EE_med, '-c', linewidth=3,  label='EE_med (SID3)')
#ax9.plot(freq_1d_sid3, sid3_EE_msk, '-b', linewidth=1,  label='EE_msk (SID3)')
ax9.plot(freq_1d_msk,  mask_list[:, 2]*1E6,   '.k', linewidth=.3,  label='mask pattern')
#
ax1.set_yscale('log');  ax2.set_yscale('log');  ax3.set_yscale('log');  ax4.set_yscale('log');  ax5.set_yscale('log')
ax6.set_yscale('log');  ax7.set_yscale('log');  ax8.set_yscale('log');  ax9.set_yscale('log')

# Label
ax1.set_ylabel(spec.str_unit); ax2.set_ylabel(spec.str_unit); ax3.set_ylabel(spec.str_unit); ax4.set_ylabel(spec.str_unit); ax5.set_ylabel(spec.str_unit)
ax6.set_ylabel(spec.str_unit); ax7.set_ylabel(spec.str_unit); ax8.set_ylabel(spec.str_unit); ax9.set_ylabel(spec.str_unit)
ax9.set_xlabel('Frequency [kHz]')
#
date1 = spec.epoch[n_sweep1];  date1 = date1.strftime(f'1st {n_sweep1}: %Y/%m/%d %R:%S   ')
date2 = spec.epoch[n_sweep2];  date2 = date2.strftime(f'Mid {n_sweep2}: %Y/%m/%d %R:%S   ')
date3 = spec.epoch[n_sweep3];  date3 = date3.strftime(f'Fin {n_sweep3}: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3
ax1.set_title(title_date)

ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8);  ax6.legend(loc='upper right', fontsize=8)
ax7.legend(loc='upper right', fontsize=8);  ax8.legend(loc='upper right', fontsize=8);  ax9.legend(loc='upper right', fontsize=8)

# range: X-axis
xlim=[f_min,       f_max];        ax1.set_xlim(xlim);  xlim=[f_min+5000,  f_max+5000];   ax2.set_xlim(xlim);  
xlim=[f_min+10000, f_max+10000];  ax3.set_xlim(xlim);  xlim=[f_min+15000, f_max+15000];  ax4.set_xlim(xlim)
xlim=[f_min+20000, f_max+20000];  ax5.set_xlim(xlim);  xlim=[f_min+25000, f_max+25000];  ax6.set_xlim(xlim)
xlim=[f_min+30000, f_max+30000];  ax7.set_xlim(xlim);  xlim=[f_min+35000, f_max+35000];  ax8.set_xlim(xlim)
xlim=[f_min+40000, f_max+40000];  ax9.set_xlim(xlim)
ax1.tick_params(labelsize=7); ax2.tick_params(labelsize=7); ax3.tick_params(labelsize=7); ax4.tick_params(labelsize=7); ax5.tick_params(labelsize=7)
ax6.tick_params(labelsize=7); ax7.tick_params(labelsize=7); ax8.tick_params(labelsize=7); ax9.tick_params(labelsize=7)

ylim=[10**p_min, 10**p_max]
ax1.set_ylim(ylim); ax2.set_ylim(ylim);  ax3.set_ylim(ylim); ax4.set_ylim(ylim); ax5.set_ylim(ylim); ax6.set_ylim(ylim);  
ax7.set_ylim(ylim); ax8.set_ylim(ylim);  ax9.set_ylim(ylim)

fig.subplots_adjust(hspace=0)
fig.show
